In [12]:
!pip3 install python-statemachine
%run helpers.ipynb
%run math_utils.ipynb

In [4]:
from statemachine import StateMachine, State
import numpy as np

In [2]:
class ADCS(StateMachine):
    """
    This class functions as the ADCS Controller.
    
    inputs = []
    outputs = []
    states = []
    
    transitions
    -----------
    
    """
    
    off = State(initial=True)
    detumbling = State()
    pointing = State()
    error = State()
    
    on_initialization = off.to(detumbling)
    to_pointing = detumbling.to(pointing)
    continuous_pointing = pointing.to(pointing)
    
    to_error = pointing.to(error)
    
    def __init__(self):
        self.moment = None
        self.pointing_vec = [0, 0, 0]
        super(ADCS, self).__init__()
        
    def b_dot(self, mag_field_vector, cang):
        detb = np.sqrt(np.dot(mag_field_vector, mag_field_vector))
        m = ((-1) / detb) * np.cross(mag_field_vector, cang)    
        return 2 * m

    def get_attitude_err(self, q_target, q_curr):
        q_err = quaternion_multiply(q_curr, quaternion_inverse(q_target))

        pitch = np.arcsin(-2*(q_err[0]*q_err[2] - q_err[1]*q_err[3]))
        roll = np.arctan(2*(q_err[1]*q_err[2] + q_err[0]*q_err[3]) / (q_err[3]**2 - q_err[0]**2 - q_err[1]**2 + q_err[2]**2))
        yaw = np.arctan(2*(q_err[0]*q_err[1] + q_err[2]*q_err[3]) / (q_err[3]**2 + q_err[0]**2 - q_err[1]**2 - q_err[2]**2))

        return np.array([roll, pitch, yaw])

    def get_attitude_rate_err(self, w_desired, w_estimated, attitude_err):
        delta_w = w_estimated - w_desired
        attitude_rate_err = -cross(w_estimated, attitude_err) + delta_w

        mag = np.linalg.norm(attitude_rate_err)
        if mag < np.finfo(np.float64).eps:
            return np.zeros(attitude_rate_err.shape)
            
        return attitude_rate_err

    def get_pointing_torque(self, q_target, q_curr, w_desired, w_estimated):
        attitude_err = self.get_attitude_err(q_target, q_curr)
        attitude_rate_err = self.get_attitude_rate_err(w_desired, w_estimated, attitude_err)
        # u = attitude_err * -0.4e-5 + attitude_rate_err * -5e-4
        u = attitude_err * -0.4e-7 + attitude_rate_err * -1e-4
        J = inertia(6, 0.34, 0.1, 0.1)
        return np.matmul(J, u)

    def get_pointing_moment(self, magnetic_field, q_target, q_curr, w_desired, w_estimated):
        torque = self.get_pointing_torque(q_target, q_curr, w_desired, w_estimated)
        torque_perp = torque - (np.dot(torque, magnetic_field)/np.dot(magnetic_field, magnetic_field)) * magnetic_field
        moment = (1/np.dot(magnetic_field, magnetic_field)) * np.cross(magnetic_field, torque_perp)
        return moment
    
    def detumble(self, mag_field, cang):
        
        # slow down rotation
        self.moment = self.b_dot(mag_field, cang)
        
    def point(self, mag_field, q_target, q_curr, w_desired, w_estimated):
        
        # update pointing vector
        
        self.moment = self.get_pointing_moment(mag_field, q_target, q_curr, w_desired, w_estimated)
    
    def get_moment(self):
        # Limit maximum moment
        # m_max = 10
        # m_mag = np.linalg.norm(self.moment)
        # if m_mag > m_max:
        #     self.moment = (m_max/m_mag) * self.moment
        return self.moment

class SafetyMonitor(StateMachine):
    
    safe = State(initial=True)
    error = State()
    
    to_error = safe.to(error, cond="moment_error")
    
    def moment_error(moment):
        return moment > 0.12
    
    def get_state(self):
        return self.current_state.id

NameError: name 'StateMachine' is not defined